In [1]:
import torch
import numpy as np
import albumentations as A
import torch.nn.functional as F

from torch.utils.data import DataLoader
from models import Tresnet_m_ml_decoder_recipy
from dataset import CustomCombineTestDataset
from utils import set_seed, custom_combine_test_collate_fn


In [2]:
torch.cuda.empty_cache()
set_seed(31)

In [3]:
tf = A.Compose(
        [
            A.Resize(224, 224),
            A.Normalize(),
        ]
    )
valid_dataset = CustomCombineTestDataset(
        "/opt/ml/level3_cv_finalproject-cv-01/model",
        tf=tf,
    )
valid_loader = DataLoader(
        dataset=valid_dataset,
        batch_size=256,
        shuffle=True,
        num_workers=8,
        collate_fn=custom_combine_test_collate_fn,
        drop_last=False,
    )

In [4]:
model = torch.load("/opt/ml/level3_cv_finalproject-cv-01/model/save/Tresnet_m_ml_decoder_recipy_latest.pth")
model.cuda()
model.eval()

FileNotFoundError: [Errno 2] No such file or directory: '/opt/ml/level3_cv_finalproject-cv-01/model/save/Tresnet_m_ml_decoder_recipy_latest.pth'

In [ ]:
with torch.no_grad():
    images_path, gt, pred = np.array([]), np.array([]), np.array([])
    flag = False
    for image_path, images, labels in valid_loader:
        images = torch.tensor(images).cuda()
        outputs = model(images)

        outputs = F.sigmoid(outputs).cpu().numpy()
        outputs = (outputs >= 0.5)
        labels = (labels >= 0.5)
        # outputs = outputs.cpu().numpy()
        if flag:
            images_path = np.concatenate((images_path, image_path), axis=0)
            gt = np.concatenate((gt, labels), axis=0)
            pred = np.concatenate((pred, outputs), axis=0)
        else:
            images_path = image_path
            gt = labels
            pred = outputs
            flag = True

In [ ]:
print(images_path.shape)
print(gt.shape)
print(pred.shape)

(18180,)
(18180, 110)
(18180, 110)


In [ ]:
import pandas as pd

data = {"image_path":[], "gt":[], "pred":[]}
for image_path, label, p in zip(images_path, gt, pred):
    data["image_path"].append(image_path)
    data["gt"].append(label)
    data["pred"].append(p)

In [ ]:
df = pd.DataFrame(data)

In [ ]:
filename = 'output_true.csv'
df.to_csv(filename, index=False)